In [1]:
import numpy as np

# ---- Raw averaged accel data from your 6 experiments (LSB) ----
raw_means = np.array([
    [16986.20,   -654.62,    183.10],   # +X
    [-15809.39,   245.21,    -20.42],   # -X
    [1080.09,   16285.10,  -338.42],   # +Y
    [597.74,  -16321.30,  1997.79],   # -Y
    [654.70,   -309.53,  17587.69],   # +Z
    [1795.17,   -435.10, -15659.34],   # -Z
])

# ---- Corresponding true unit vectors (in g) ----
true_vecs = np.array([
    [ 1,  0,  0],   # +X
    [-1,  0,  0],   # -X
    [ 0,  1,  0],   # +Y
    [ 0, -1,  0],   # -Y
    [ 0,  0,  1],   # +Z
    [ 0,  0, -1],   # -Z
])

# ---- Solve for affine transform: a_true = M * (a_raw - b) ----

# Build linear system: A * [M.flatten(); b] ≈ y
# Expand each equation: a_true = M * a_raw + c
N = raw_means.shape[0]
A = []
Y = []
for i in range(N):
    xr, yr, zr = raw_means[i]
    xt, yt, zt = true_vecs[i]

    # Each true component gives one row
    # Row for x_true
    A.append([xr, yr, zr, 0, 0, 0, 0, 0, 0, 1, 0, 0])
    Y.append(xt)

    # Row for y_true
    A.append([0, 0, 0, xr, yr, zr, 0, 0, 0, 0, 1, 0])
    Y.append(yt)

    # Row for z_true
    A.append([0, 0, 0, 0, 0, 0, xr, yr, zr, 0, 0, 1])
    Y.append(zt)

A = np.array(A)
Y = np.array(Y)

# Least squares solution
params, _, _, _ = np.linalg.lstsq(A, Y, rcond=None)

# Extract M and b
M = params[:9].reshape(3, 3)
b = params[9:].reshape(3,)


# Conversion to Physical units
ACCEL_SENS = 16384.0  # LSB/g for ±2g


print("Calibration results:")
print("M =\n", M)
print("b =\n", b)

# ---- Test: apply calibration to your raw points ----
for i, raw in enumerate(raw_means):
    corrected = M.dot(raw - b) / ACCEL_SENS
    print(f"Raw {raw} → Corrected {corrected} (expected {true_vecs[i]})")

# Optional: print calibration matrices
print("\nCalibration matrix M (affine):\n", M)
print("Bias vector b (LSB):\n", b)


Calibration results:
M =
 [[ 6.08997382e-05 -7.41227306e-07  2.03088121e-06]
 [ 1.69669384e-06  6.12865742e-05 -1.71520618e-07]
 [-3.16142589e-07  4.30764171e-06  6.00314277e-05]]
b =
 [-0.05525702  0.01076481 -0.03638963]
Raw [16986.2   -654.62   183.1 ] → Corrected [ 6.31906529e-05 -6.91590204e-07  1.71139248e-07] (expected [1 0 0])
Raw [-15809.39    245.21    -20.42] → Corrected [-5.87773127e-05 -7.19768285e-07  2.94835036e-07] (expected [-1  0  0])
Raw [ 1080.09 16285.1   -338.42] → Corrected [3.23623008e-06 6.10319852e-05 3.02094717e-06] (expected [0 1 0])
Raw [   597.74 -16321.3    1997.79] → Corrected [ 3.20805200e-06 -6.10110849e-05  3.01739642e-06] (expected [ 0 -1  0])
Raw [  654.7   -309.53 17587.69] → Corrected [ 4.62783489e-06 -1.27419626e-06  6.43478967e-05] (expected [0 0 1])
Raw [  1795.17   -435.1  -15659.34] → Corrected [ 4.75153068e-06 -1.27774701e-06 -5.75251588e-05] (expected [ 0  0 -1])

Calibration matrix M (affine):
 [[ 6.08997382e-05 -7.41227306e-07  2.03088121

In [2]:
import numpy as np

# ---- Raw averaged accel data from 6 orientations (LSB) ----
raw_accel = np.array([
    [16986.20,   -654.62,    183.10],   # +X
    [-15809.39,   245.21,    -20.42],   # -X
    [1080.09,   16285.10,  -338.42],    # +Y
    [597.74,  -16321.30,  1997.79],     # -Y
    [654.70,   -309.53,  17587.69],     # +Z
    [1795.17,   -435.10, -15659.34],    # -Z
])

# ---- Raw averaged gyro data (LSB) ----
raw_gyro = np.array([
    [-555.51, 112.13, -71.25],   # +X
    [-559.62, 118.84, -66.77],   # -X
    [-573.55, 91.17, -82.27],    # +Y
    [-565.19, 107.73, -77.63],   # -Y
    [-558.03, 111.44, -71.00],   # +Z
    [-556.80, 108.93, -72.50],   # -Z
])

# ---- True unit vectors (g) for accelerometer ----
true_accel = np.array([
    [ 1,  0,  0], [ -1, 0, 0],
    [ 0,  1,  0], [ 0, -1, 0],
    [ 0,  0,  1], [ 0,  0, -1]
])

# ---- Solve affine calibration: a_true = M*(a_raw - b) ----
N = raw_accel.shape[0]
A = []
Y = []
for i in range(N):
    xr, yr, zr = raw_accel[i]
    xt, yt, zt = true_accel[i]
    # x_true row
    A.append([xr, yr, zr, 0, 0, 0, 0, 0, 0, 1, 0, 0])
    Y.append(xt)
    # y_true row
    A.append([0, 0, 0, xr, yr, zr, 0, 0, 0, 0, 1, 0])
    Y.append(yt)
    # z_true row
    A.append([0, 0, 0, 0, 0, 0, xr, yr, zr, 0, 0, 1])
    Y.append(zt)

A = np.array(A)
Y = np.array(Y)
params, _, _, _ = np.linalg.lstsq(A, Y, rcond=None)

# Extract M (3x3) and bias vector b
M = params[:9].reshape(3, 3)
b = params[9:].reshape(3,)

print("=== Accelerometer Calibration ===")
print("Calibration matrix M (maps raw -> corrected counts):\n", M)
print("Bias vector b (LSB):", b)

# ---- Sensitivity conversions ----
ACCEL_SENS = 16384.0  # LSB/g for ±2g
GYRO_SENS = 131.0     # LSB/(°/s) for ±250 dps

# ---- Apply calibration to accelerometer and convert to g ----
print("\nCalibrated accelerometer readings in g (after applying M and b):")
for i, raw in enumerate(raw_accel):
    corrected_counts = M.dot(raw - b)      # still in LSB
    corrected_g = corrected_counts / ACCEL_SENS
    print(f"Raw {raw} -> Corrected {corrected_g} g (expected {true_accel[i]} g)")

# ---- Gyroscope bias subtraction and conversion to °/s ----
gyro_bias = np.mean(raw_gyro, axis=0)
print("\nGyro bias (LSB):", gyro_bias)
print("Calibrated gyro readings in °/s:")
for i, raw in enumerate(raw_gyro):
    corrected_dps = (raw - gyro_bias) / GYRO_SENS
    print(f"Raw {raw} -> Corrected {corrected_dps} °/s (expected ~0 stationary)")


=== Accelerometer Calibration ===
Calibration matrix M (maps raw -> corrected counts):
 [[ 6.08997382e-05 -7.41227306e-07  2.03088121e-06]
 [ 1.69669384e-06  6.12865742e-05 -1.71520618e-07]
 [-3.16142589e-07  4.30764171e-06  6.00314277e-05]]
Bias vector b (LSB): [-0.05525702  0.01076481 -0.03638963]

Calibrated accelerometer readings in g (after applying M and b):
Raw [16986.2   -654.62   183.1 ] -> Corrected [ 6.31906529e-05 -6.91590204e-07  1.71139248e-07] g (expected [1 0 0] g)
Raw [-15809.39    245.21    -20.42] -> Corrected [-5.87773127e-05 -7.19768285e-07  2.94835036e-07] g (expected [-1  0  0] g)
Raw [ 1080.09 16285.1   -338.42] -> Corrected [3.23623008e-06 6.10319852e-05 3.02094717e-06] g (expected [0 1 0] g)
Raw [   597.74 -16321.3    1997.79] -> Corrected [ 3.20805200e-06 -6.10110849e-05  3.01739642e-06] g (expected [ 0 -1  0] g)
Raw [  654.7   -309.53 17587.69] -> Corrected [ 4.62783489e-06 -1.27419626e-06  6.43478967e-05] g (expected [0 0 1] g)
Raw [  1795.17   -435.1  -156